# Dự án: Gom nhóm dữ liệu bằng các phương pháp học máy

Chạy toàn bộ quy trình: tiền xử lý, phân cụm (K-Means tự viết, K-Means thư viện, DBSCAN), đánh giá và trực quan hóa.

In [1]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
from src.preprocessing import preprocess_data
from src.kmeans_custom import kmeans_custom, compute_inertia
from src.kmeans_library import kmeans_library, compute_inertia_library
from src.dbscan_library import dbscan_library
from src.evaluation import evaluate_clustering, save_evaluation_results
from src.visualization import plot_clusters

## 1. Tiền xử lý dữ liệu

In [2]:
# Tiền xử lý Mall Customer Segmentation Data
X_scaled_df, X_pca_df = preprocess_data(
    file_path="data/Mall_Customers.csv",
    output_path_scaled="data/processed_mall_customers.csv",
    output_path_pca="data/mall_customers_pca.csv",
    use_pca=True,
    n_components=2
)

# Chuyển thành numpy array
X_scaled = X_scaled_df.values
X_pca = X_pca_df.values

Dữ liệu đã chuẩn hóa được lưu tại: data/processed_mall_customers.csv
Dữ liệu sau PCA được lưu tại: data/mall_customers_pca.csv
Tỷ lệ phương sai được giải thích bởi PCA: [0.44266167 0.33308378]


## 2. Phân cụm

In [3]:
# Tham số
k = 5  # Số cụm cho K-Means
eps = 0.5  # Bán kính cho DBSCAN
min_samples = 5  # Số điểm tối thiểu cho DBSCAN

# K-Means tự viết
labels_kmeans_custom, centroids_kmeans_custom = kmeans_custom(X_scaled, k=k, random_state=42)
inertia_custom = compute_inertia(X_scaled, labels_kmeans_custom, centroids_kmeans_custom)
print(f"K-Means Custom - Inertia: {inertia_custom:.4f}")

# K-Means thư viện
labels_kmeans_library, centroids_kmeans_library = kmeans_library(X_scaled, k=k, random_state=42)
inertia_library = compute_inertia_library(X_scaled, labels_kmeans_library, centroids_kmeans_library)
print(f"K-Means Library - Inertia: {inertia_library:.4f}")

# DBSCAN
labels_dbscan = dbscan_library(X_scaled, eps=eps, min_samples=min_samples)

K-Means Custom - Inertia: 168.2476
K-Means Library - Inertia: 169.8915


c:\Users\vkhoa\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


## 3. Đánh giá

In [4]:
# Đánh giá các phương pháp
evaluation_results = {}
evaluation_results["K-Means Custom"] = evaluate_clustering(X_scaled, labels_kmeans_custom, "K-Means Custom")
evaluation_results["K-Means Library"] = evaluate_clustering(X_scaled, labels_kmeans_library, "K-Means Library")
evaluation_results["DBSCAN"] = evaluate_clustering(X_scaled, labels_dbscan, "DBSCAN")

# Lưu kết quả đánh giá
save_evaluation_results(evaluation_results, output_path="results/evaluation_metrics.txt")

K-Means Custom - Silhouette Score: 0.4166, Davies-Bouldin Index: 0.8746
K-Means Library - Silhouette Score: 0.4085, Davies-Bouldin Index: 0.8890
DBSCAN - Silhouette Score: 0.1845, Davies-Bouldin Index: 1.7569
Kết quả đánh giá đã lưu tại: results/evaluation_metrics.txt


## 4. Trực quan hóa

In [5]:
# Trực quan hóa (dùng dữ liệu sau PCA để vẽ 2D)
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(X_scaled)

plot_clusters(
    X=X_pca,
    labels=labels_kmeans_custom,
    centroids=pca.transform(centroids_kmeans_custom),
    method_name="K-Means Custom",
    save_path="results/kmeans_custom_plot.png"
)

plot_clusters(
    X=X_pca,
    labels=labels_kmeans_library,
    centroids=pca.transform(centroids_kmeans_library),
    method_name="K-Means Library",
    save_path="results/kmeans_library_plot.png"
)

plot_clusters(
    X=X_pca,
    labels=labels_dbscan,
    centroids=None,  # DBSCAN không có tâm cụm
    method_name="DBSCAN",
    save_path="results/dbscan_plot.png"
)

Biểu đồ đã lưu tại: results/kmeans_custom_plot.png
Biểu đồ đã lưu tại: results/kmeans_library_plot.png
Biểu đồ đã lưu tại: results/dbscan_plot.png


c:\Khoa\Y3_HK2\MachineLearning\Clustering_Project\src\visualization.py:23: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()
